In [1]:
import pandas as pd
import numpy as np

NB: There is some numpy wonkiness going on behind the scenes here. You'll see it if you look at the data types of the data frames. For some reason, numpy infers the data type as a float for any column that contains an NaN. We can't force the issue by specifying dtype='int' in read_csv, because then numpy throws an error when it can't convert the NaN to an int.

I didn't drop the NaN values, because we won't be keeping all of these columns, and I didn't want to prematurely prune the dataset.

Obviously we don't want floats for categorical variables, but I don't think this will be an issue since we'll be using get_dummies to generate the variables. Worse come to worst, we'll just have to cast all float data points to ints using to_numeric or something like that, which will be annoying, but not the end of the world.

Let me know if you spot a workaround I didn't think of.

#### Import biomarkers

In [2]:
glucHome = pd.read_csv(r'C:\dev\data\health\dataSets\glucoseHomeostasis.tsv', sep='\t', header=0, na_values=' ')
infImmFunc = pd.read_csv(r'C:\dev\data\health\dataSets\inflammationAndImmuneFunction.tsv', sep='\t', header=0, na_values=' ')
lipids = pd.read_csv(r'C:\dev\data\health\dataSets\lipids.tsv', sep='\t', header=0, na_values=' ')

In [3]:
# glucHome.head()

In [4]:
glucHome.shape

(5114, 10)

In [5]:
# Note EBV_FLAG. We probably aren't going to use these, but if we do, the NaN need to be filled in with zeros
lipids.head()

,AID,TG,TG_FLG,TC,TC_FLG,HDL,HDL_FLG,LDL,NON_HDL,TC_HDL,C_MED2,C_JOINT2,FASTTIME,FAST_LIP
0,57101310,7.0,1.0,8.0,2.0,3.0,2.0,9.0,9.0,9.0,0,1,1.6,0.0
1,57103869,99.0,9.0,99.0,9.0,99.0,9.0,99.0,99.0,99.0,0,0,5.3,0.0
2,57109625,7.0,1.0,8.0,2.0,3.0,2.0,8.0,9.0,9.0,0,0,2.3,0.0
3,57111071,4.0,2.0,7.0,2.0,4.0,2.0,8.0,7.0,8.0,0,0,2.4,0.0
4,57113943,99.0,9.0,1.0,2.0,99.0,9.0,99.0,99.0,99.0,0,0,2.7,0.0


In [6]:
infImmFunc.shape

(5114, 16)

In [7]:
'''We probably won't use these flags, but filling in NaNs with zero. 
I did these two because they were fast, but there are a lot of them scattered
throughout the In-Home Questionnaire sets. Something to keep an eye out for.'''
infImmFunc['CRP_FLAG'].fillna(0, inplace=True, downcast='int')
infImmFunc['EBV_FLAG'].fillna(0, inplace=True, downcast='int')
infImmFunc.head()

,AID,CRP,CRP_FLAG,EBV,EBV_FLAG,C_CRP,C_SUBCLN,C_INFECT,CRP_MED1,CRP_MED2,CRP_MED3,CRP_MED4,CRP_MED5,CRP_MED6,CRP_MED7,CRP_MED8
0,57101310,8.448,0,90.0,0,3.0,2,0,1,0,0,0,0,0,0,1
1,57103869,999.000,0,9999.0,0,9.0,0,0,0,0,0,0,0,0,0,0
2,57109625,1.204,0,187.0,0,2.0,1,0,0,0,0,0,0,0,0,0
3,57111071,0.905,0,75.0,0,1.0,0,0,0,0,0,0,0,0,0,0
4,57113943,5.363,0,257.0,0,3.0,3,1,1,0,0,0,0,0,0,1


In [8]:
# lipids.head()

In [9]:
lipids.shape

(5114, 14)

#### Importing Waves

NB: Wave 4 comes in mutiple files. I included only the main In-Home Questionaire file. 16b and 16c contain question re. intimate partners. I did not include it in light of previous dicussions, but it can easily be brought in. I also did not include section 18, 19 and 20 which contains data on pregancies, live births and children per our discussion. We can easily include if minds change.

In [10]:
wave1 = pd.read_csv(r'C:\dev\data\health\dataSets\wave1_inHomeQuest.tsv', sep='\t', header=0, na_values=' ')
wave2 = pd.read_csv(r'C:\dev\data\health\dataSets\wave2_inHomeQuest.tsv', sep='\t', header=0, na_values=' ')
wave3 = pd.read_csv(r'C:\dev\data\health\dataSets\wave3_inHomeQuest.tsv', sep='\t', header=0, na_values=' ')
wave4 = pd.read_csv(r'C:\dev\data\health\dataSets\wave4_inHomeQuest.tsv', sep='\t', header=0, na_values=' ')

In [11]:
# wave1.head()

In [12]:
wave1.shape

(6504, 2794)

In [13]:
# Note flags
# wave2.head()

In [14]:
wave2.shape

(4834, 2532)

In [15]:
# wave3.head()

In [16]:
wave3.shape

(4882, 1831)

In [17]:
# wave4.head()

In [18]:
wave4.shape

(5114, 920)

#### Importing Public Use Contextual Databases

NB: These datasets contain rich demographic information for individuals. They only exist for Waves 1 and 2

In [19]:
wave1_publicUse = pd.read_csv(r'C:\dev\data\health\dataSets\wave1_publicUseContDB.tsv', sep='\t', header=0, na_values=' ')
wave2_publicUse = pd.read_csv(r'C:\dev\data\health\dataSets\wave2_publicUseContDB.tsv', sep='\t', header=0, na_values=' ')

In [20]:
# wave1_publicUse.head()

In [21]:
wave1_publicUse.shape

(6504, 32)

In [22]:
# wave2_publicUse.head()

In [23]:
wave2_publicUse.shape

(4834, 32)

#### Merging Waves 1 and 2 with Public Use Contextual Databases

In [24]:
wave1 = pd.merge(left=wave1, right=wave1_publicUse, left_on='AID', right_on='AID')
wave2 = pd.merge(left=wave2, right=wave2_publicUse, left_on='AID', right_on='AID')

In [25]:
# wave1.head()

In [26]:
wave1.shape

(6504, 2825)

In [27]:
# wave2.head()

In [28]:
wave2.shape

(4834, 2563)

#### Merging Waves with Biomarkers

NB: I created 3 merged datasets for each wave, one for each set of biomarkers. Suspect we'll be working discretely with different biomarkers. This way we can just pair down the dataset to desired variables directly in one go. If you want a single dataset with all biomarkers it'll be quick to do.

In [29]:
wave1_gluc = pd.merge(left=wave1, right=glucHome, left_on='AID', right_on='AID')
wave1_infImm = pd.merge(left=wave1, right=infImmFunc, left_on='AID', right_on='AID')
wave1_lipids = pd.merge(left=wave1, right=lipids, left_on='AID', right_on='AID')

In [30]:
print wave1_gluc.shape
print wave1_infImm.shape
print wave1_lipids.shape

(5114, 2834)
(5114, 2840)
(5114, 2838)


In [31]:
wave2_gluc = pd.merge(left=wave2, right=glucHome, left_on='AID', right_on='AID')
wave2_infImm = pd.merge(left=wave2, right=infImmFunc, left_on='AID', right_on='AID')
wave2_lipids = pd.merge(left=wave2, right=lipids, left_on='AID', right_on='AID')

In [32]:
# wave2_gluc.head()

In [33]:
# wave2_infImm.head()

In [34]:
# wave2_lipids.head()

In [35]:
print wave2_gluc.shape
print wave2_infImm.shape
print wave2_lipids.shape

(3924, 2572)
(3924, 2578)
(3924, 2576)


In [36]:
wave3_gluc = pd.merge(left=wave3, right=glucHome, left_on='AID', right_on='AID')
wave3_infImm = pd.merge(left=wave3, right=infImmFunc, left_on='AID', right_on='AID')
wave3_lipids = pd.merge(left=wave3, right=lipids, left_on='AID', right_on='AID')

In [37]:
# wave3_gluc.head()

In [38]:
# wave3_infImm.head()

In [39]:
# wave3_lipids.head()

In [40]:
print wave3_gluc.shape
print wave3_infImm.shape
print wave3_lipids.shape

(4208, 1840)
(4208, 1846)
(4208, 1844)


In [41]:
wave4_gluc = pd.merge(left=wave4, right=glucHome, left_on='AID', right_on='AID')
wave4_infImm = pd.merge(left=wave4, right=infImmFunc, left_on='AID', right_on='AID')
wave4_lipids = pd.merge(left=wave4, right=lipids, left_on='AID', right_on='AID')

In [42]:
# wave4_gluc.head()

In [43]:
# wave4_infImm.head()

In [44]:
# wave4_lipids.head()

In [45]:
print wave4_gluc.shape
print wave4_infImm.shape
print wave4_lipids.shape

(5114, 929)
(5114, 935)
(5114, 933)


In [46]:
def filterDf(df):

    #get fields we manually selected from wave 1
    sf = pd.read_csv(r'C:\dev\data\health\dataSets\wave1_selectedFields.csv')
    selectedFields = set(sf.Variable)
    
    #later we might want to translate the manually selected fields to equivalents in waves 2, 3, 4

    #for now, keep all the biomarkers
    selectedFields = selectedFields.union(["C_JOINT"])
    selectedFields = selectedFields.union(["C_CRP"])
    selectedFields = selectedFields.union(["C_JOINT2"])
    
    #drop columns with many NaN values
    selectedFields = selectedFields.difference(['CONVERT','C_HBA1C','HBA1C','C_FGLU','C_NFGLU', 'GLUCOSE','FASTTIME'])
    
    #and keep all the publicUse stuff
    selectedFields = selectedFields.union(wave1_publicUse.columns)
    selectedFields = selectedFields.union(wave2_publicUse.columns)
    
    #wildcard matches wave 1 section 1
    wildCardPatterns = ['H1GI']
    
    slctdCols = filter(lambda x: x in selectedFields or any(y in x for y in wildCardPatterns), df.columns)
    
    return df[slctdCols].copy()

In [47]:
"C_JOINT2" in wave1_lipids.columns

# for col in wave1_infImm.columns:
#     print col

True

In [48]:
wave1_gluc_filt = filterDf(wave1_gluc)
wave1_imm_filt = filterDf(wave1_infImm)
wave1_lip_filt = filterDf(wave1_lipids)

In [74]:
def dropNa(df):

    floatCols = []

    for col in df.columns:
        if df[col].dtype == np.float64:
            floatCols.append(col)

    nullIdxs = set()
    colCnts = dict()

    for i in range(len(df)):
        row = df.iloc[i]
        for col in floatCols:
            val = row[col]
            if np.isnan(val):
                nullIdxs.add(i)
                if col not in colCnts:
                    colCnts[col] = 0
                colCnts[col] += 1

    print("Total rows to drop: %d" % len(nullIdxs))

    for col in colCnts:
        if colCnts[col] > 5:
            print(col, colCnts[col])
        
    return df.drop(df.index[list(nullIdxs)]).copy()
    

In [51]:
print("gluc")
wave1_gluc_filt_clean = dropNa(wave1_gluc_filt)

print("imm")
wave1_imm_filt_clean = dropNa(wave1_imm_filt)

print("lip")
wave1_lip_filt_clean = dropNa(wave1_lip_filt)

gluc
Total rows to drop: 7
('H1TO39', 1)
('H1GI3', 1)
('H1TO42', 1)
('H1FV13', 1)
('H1TO38', 1)
('H1NM9', 1)
('H1NM8', 1)
('H1EE6', 1)
('H1TO41', 1)
('H1TO9', 1)
imm
Total rows to drop: 398
('H1TO39', 1)
('H1GI3', 1)
('H1TO42', 1)
('H1FV13', 1)
('C_CRP', 391)
('H1TO38', 1)
('H1NM9', 1)
('H1NM8', 1)
('H1EE6', 1)
('H1TO41', 1)
('H1TO9', 1)
lip
Total rows to drop: 7
('H1TO39', 1)
('H1GI3', 1)
('H1TO42', 1)
('H1FV13', 1)
('H1TO38', 1)
('H1NM9', 1)
('H1NM8', 1)
('H1EE6', 1)
('H1TO41', 1)
('H1TO9', 1)


In [52]:
wave1_gluc_filt_clean.to_csv(r'C:\dev\data\health\dataSets\wave1_gluc_clean.csv')
wave1_imm_filt_clean.to_csv(r'C:\dev\data\health\dataSets\wave1_imm_clean.csv')
wave1_lip_filt_clean.to_csv(r'C:\dev\data\health\dataSets\wave1_lip_clean.csv')

In [53]:
print(len(wave1_gluc_filt_clean))
print(len(wave1_imm_filt_clean))
print(len(wave1_lip_filt_clean))

5107
4716
5107


In [66]:
def filter4(df):
    slctdCols = filter(lambda x: not any(y in x for y in ['H4RD','H4IR','H4EO']), df.columns)

    dropCols = ['CONVERT','C_HBA1C','HBA1C','C_FGLU','C_NFGLU', 'GLUCOSE','FASTTIME','H4DS20','H4DS18','H4DS19','H4DS16','H4DS17',
                'H4DS14','H4DS15','H4LM17','H4DS13','PTNR_ID']

    slctdCols = frozenset(slctdCols).difference(dropCols)

    return df[list(slctdCols)].copy()


In [67]:
wave4_gluc_filt = filter4(wave4_gluc)
wave4_imm_filt = filter4(wave4_infImm)
wave4_lip_filt = filter4(wave4_lipids)

In [69]:
wave4_gluc.shape

(5114, 929)

In [70]:
print("gluc")
wave4_gluc_filt_clean = dropNa(wave4_gluc_filt)

print("imm")
wave4_imm_filt_clean = dropNa(wave4_imm_filt)

print("lip")
wave4_lip_filt_clean = dropNa(wave4_lip_filt)

gluc
Total rows to drop: 131
('H4DA2', 1)
('H4GH3Y', 11)
('H4PE5', 1)
('H4CUFF', 43)
('H4LM19', 2)
('H4PE8', 1)
('H4PE9', 1)
('H4PE6', 1)
('H4PE7', 1)
('H4PE4', 1)
('H4MAP', 43)
('H4PE2', 1)
('H4PE3', 1)
('H4LM12', 2)
('H4PE1', 1)
('H4CJ8', 1)
('H4DA6', 1)
('H4DA7', 1)
('H4PE41', 1)
('H4PE40', 1)
('C4WD90_2', 2)
('C4WD90_3', 2)
('C4WD90_1', 2)
('H4DA5', 1)
('H4LM14', 2)
('H4PE29', 1)
('H4DA17', 1)
('H4PE28', 1)
('H4SBP', 43)
('H4SE37O', 1)
('H4LM15Y', 2)
('H4SE37H', 1)
('H4GH4B', 1)
('H4HR11YN', 1)
('H4DA1', 1)
('H4GH4A', 1)
('H4PE25', 1)
('C4WD60_3', 6)
('C4WD60_2', 6)
('C4WD60_1', 6)
('H4PE24', 1)
('H4LM11', 2)
('H4ARM', 43)
('H4DA25', 1)
('H4DA24', 1)
('H4DA27', 3)
('H4DA26', 3)
('H4DA21', 3)
('H4DA20', 3)
('H4DA23', 3)
('H4DA22', 3)
('H4PE21', 1)
('H4HR7J', 6)
('H4GH3D', 11)
('H4PE20', 1)
('H4SE5', 2)
('H4SE4', 2)
('H4SE1', 2)
('H4DA8', 1)
('H4PE38', 1)
('H4PE39', 1)
('H4PE36', 1)
('H4PE37', 1)
('H4PE34', 1)
('H4DA12T', 1)
('H4PE32', 1)
('H4PE33', 1)
('H4PE30', 1)
('H4PE31', 1)
('H

In [71]:
def countDtypes(df):
    dTypes = dict()
    for col in df.columns:
        key = str(df[col].dtype)
        if key not in dTypes:
            dTypes[key] = []
        dTypes[key].append(col)
#     print(dTypes)
    return dTypes


In [77]:
def dummy(df):
    z = df
    
    z = z[z.columns.drop(countDtypes(z)['object'])]

    toDummy = []

    for col in z:
        vals = z[col].unique()

        if len(vals) > 10:
#             print("%8s - continuous? %s" % (col, z[col].dtype))
            continue
        else:
#             print("%8s - %s" % (col, vals))
            toDummy.append(col)

    toDummy = filter(lambda x: x not in ['C_JOINT','C_CRP','C_JOINT2'], toDummy)
    
    dum = pd.get_dummies(z, columns=toDummy)
    
    colsToDrop = []

    for col in dum.columns:
        split = str.split(col, "_")

        key = split[-1:][0]

        try:
    #         print(key)
            n = float(key)

            if n > 7:
                colsToDrop.append(col)
        except:
            continue
    
    return dum[dum.columns.drop(colsToDrop)].copy()

In [78]:
wave4_gluc_dum = dummy(wave4_gluc_filt_clean)
wave4_imm_dum = dummy(wave4_imm_filt_clean)
wave4_lip_dum = dummy(wave4_lip_filt_clean)

In [80]:
wave4_gluc_dum.to_csv(r'C:\dev\data\health\dataSets\wave4_gluc_dummies.csv')
wave4_imm_dum.to_csv(r'C:\dev\data\health\dataSets\wave4_imm_dummies.csv')
wave4_lip_dum.to_csv(r'C:\dev\data\health\dataSets\wave4_lip_dummies.csv')

In [79]:
wave4_gluc_dum.shape

(4983, 2649)

In [ ]:
slctdCols = filter(lambda x: not any(y in x for y in ['H4RD','H4IR','H4EO']), wave4_gluc.columns)

dropCols = ['CONVERT','C_HBA1C','HBA1C','C_FGLU','C_NFGLU', 'GLUCOSE','FASTTIME','H4DS20','H4DS18','H4DS19','H4DS16','H4DS17',
            'H4DS14','H4DS15','H4LM17','H4DS13','PTNR_ID']

slctdCols = frozenset(slctdCols).difference(dropCols)

wave4_gluc_clean = wave4_gluc[list(slctdCols)].copy()

wave4_gluc_clean.shape

In [ ]:
wave4_gluc_dum = pd.get_dummies(z, columns=toDummy)
wave4_gluc_dum.shape

In [ ]:
z = dropNa(df)

In [ ]:
colsToDrop = []

for col in wave4_gluc_dum.columns:
    split = str.split(col, "_")
    
    key = split[-1:][0]
    
    try:
#         print(key)
        n = float(key)
        
        if n > 7:
            colsToDrop.append(col)
    except:
        continue

colsToDrop = colsToDrop
len(colsToDrop)

In [ ]:
wave4_gluc_dum1 = wave4_gluc_dum[wave4_gluc_dum.columns.drop(colsToDrop)].copy()
wave4_gluc_dum1.shape